In [1]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
from optuna import Trial, visualization



torch.__version__

g:\My Drive\Kaggle\tabular-playground-series-apr-2022\tab22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.10.1+cu113'

In [2]:
!nvidia-smi

Tue Apr 12 09:02:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   39C    P5    20W / 250W |    938MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data = pd.read_csv('../Data/train_features_computed_tabular.csv')
labels = pd.read_csv('../Data/train_labels.csv')
data = data.merge(labels, on = 'sequence', how = 'left')
X, y = data.drop(['sequence', 'state'], axis = 1).values, data['state'].values

In [4]:
scaler = StandardScaler()
model=scaler.fit(X)
X=model.transform(X)

In [5]:
def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1.0, 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    param = dict(
        n_d=n_da,
        n_a=n_da,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        mask_type=mask_type,
        n_shared=n_shared,
        scheduler_params=dict(
            mode="min",
            patience=trial.suggest_int("patienceScheduler", low=3, high=10),
            min_lr=1e-5,
            factor=0.5,
        ),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose=0,
    )
    kf = KFold(n_splits=3, random_state=42, shuffle=True)
    CV_score_array = []
    for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        clf = TabNetClassifier(**param)
        clf.fit(
            X_train=X_train,
            y_train=y_train,
            eval_set=[(X_valid, y_valid)],
            patience=trial.suggest_int("patience", low=15, high=30),
            max_epochs=trial.suggest_int("epochs", 1, 100),
        )
        preds = clf.predict(X_valid)
        auc = roc_auc_score(y_valid, preds)

        CV_score_array.append(auc)
    avg = np.mean(CV_score_array)
    return avg


In [6]:
study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
study.optimize(Objective, timeout=6*60) #5 hours

best = study.best_params
print('The best parameters are ', best)

[I 2022-04-12 09:02:40,017] A new study created in memory with name: TabNet optimization


Stop training because you reached max_epochs = 36 with best_epoch = 24 and best_val_0_auc = 0.87029
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 36 with best_epoch = 27 and best_val_0_auc = 0.87781
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 36 with best_epoch = 27 and best_val_0_auc = 0.86642
Best weights from best epoch are automatically used!


[I 2022-04-12 09:04:28,231] Trial 0 finished with value: 0.7918052503059556 and parameters: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.2, 'n_shared': 3, 'lambda_sparse': 0.00023593938579518834, 'patienceScheduler': 8, 'patience': 17, 'epochs': 36}. Best is trial 0 with value: 0.7918052503059556.


Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_val_0_auc = 0.8251
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_val_0_auc = 0.82409
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_val_0_auc = 0.82288
Best weights from best epoch are automatically used!


[I 2022-04-12 09:04:32,012] Trial 1 finished with value: 0.7579470838729235 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 0.0001773258467273771, 'patienceScheduler': 6, 'patience': 19, 'epochs': 2}. Best is trial 0 with value: 0.7918052503059556.


...

Stop training because you reached max_epochs = 62 with best_epoch = 57 and best_val_0_auc = 0.87391
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 52 with best_epoch = 23 and best_val_0_auc = 0.87596
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 62 with best_epoch = 55 and best_val_0_auc = 0.87098
Best weights from best epoch are automatically used!


[I 2022-04-12 09:07:00,427] Trial 3 finished with value: 0.793408361234848 and parameters: {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 2, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 2.5693078926127237e-05, 'patienceScheduler': 6, 'patience': 29, 'epochs': 62}. Best is trial 3 with value: 0.793408361234848.



Early stopping occurred at epoch 57 with best_epoch = 39 and best_val_0_auc = 0.83273
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 77 with best_epoch = 59 and best_val_0_auc = 0.84699
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 63 with best_epoch = 45 and best_val_0_auc = 0.84083
Best weights from best epoch are automatically used!


[I 2022-04-12 09:09:46,573] Trial 4 finished with value: 0.7667660875603742 and parameters: {'mask_type': 'sparsemax', 'n_da': 60, 'n_steps': 3, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 2.2050446144967754e-06, 'patienceScheduler': 3, 'patience': 18, 'epochs': 97}. Best is trial 3 with value: 0.793408361234848.


The best parameters are  {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 2, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 2.5693078926127237e-05, 'patienceScheduler': 6, 'patience': 29, 'epochs': 62}


In [7]:
assert False # not run 
best_model_params = dict(
    n_d=best["n_da"],
    n_a=best["n_da"],
    n_steps=best["n_steps"],
    gamma=best["gamma"],
    lambda_sparse=best["lambda_sparse"],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type=best["mask_type"],
    n_shared=best["n_shared"],
    scheduler_params=dict(
        mode="min",
        patience=best["patienceScheduler"],
        min_lr=1e-5,
        factor=0.5,
    ),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=0,
)
epochs = best["epochs"]

clf = TabNetClassifier(**best_model_params)
clf.fit(X_train=X, y_train=y,
          patience=best['patience'], max_epochs=epochs,
          eval_metric=['auc'])


AssertionError: 